In [1]:
import os
import sys
sys.path.append("..")

# -- network analysis (later install graph-tool)
import pandas as pd
import geopandas as gpd

import networkx as nx
import matplotlib.pyplot as plt

from fluxsus.fluxnets import FluxNets

PyTables is not installed. No support for HDF output.


In [2]:
datapath = os.path.join(os.environ["HOMEPATH"], "Documents", "data")
sihpath = os.path.join(datapath, "opendatasus", "sihsus", "PARQUET_ANOS")
geopath = os.path.join(datapath, "shapefilesceqgis")

In [51]:
# -- SIHSUS
year_range = range(2017, 2022)

sihsus_df = []
for year in year_range:
    sihsus_df.append( pd.read_parquet(os.path.join(sihpath, f"RDCE_{year}.parquet")) )
sihsus_df = pd.concat(sihsus_df)

In [52]:
# -- GEO DATA
mun_df = gpd.read_file( os.path.join(geopath, "Ceará MUN.shp") )
macro_df = gpd.read_file( os.path.join(geopath, "Macro Ceará.shp") )

macro_df["ID"] = macro_df["ID"].astype(int)
mun_df["GEO6"] = mun_df["GEO6"].astype(str)

mun_df1 = mun_df.merge(macro_df[["ID", "Macro Shee"]], left_on="MACRO", right_on="ID").drop("ID", axis=1).rename({"Macro Shee": "Macro Nome"}, axis=1)
mun_df1["centroid"] = mun_df1.centroid
mun_df1['lon'] = mun_df1['centroid'].apply(lambda x: x.x)
mun_df1['lat'] = mun_df1['centroid'].apply(lambda x: x.y)
mun_df1 = mun_df1.drop('centroid', axis=1)
mun_df1.head()

C:\Users\higor.monteiro\AppData\Local\Temp\ipykernel_22840\3092562899.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mun_df1["centroid"] = mun_df1.centroid


,CD_GEOCODM,NM_MUNICIP,GEO6,MACRO,CRES,geometry,Macro Nome,lon,lat
0,2300101,ABAIARA,230010,5,19,"POLYGON ((-39.08246 -7.29577, -39.08347 -7.292...",Superintendência Regional de Saúde Cariri,-39.037538,-7.359907
1,2300309,ACOPIARA,230030,5,18,"POLYGON ((-39.21390 -5.99175, -39.21278 -5.991...",Superintendência Regional de Saúde Cariri,-39.492823,-6.122411
2,2300606,ALTANEIRA,230060,5,20,"POLYGON ((-39.70288 -6.97367, -39.69772 -6.961...",Superintendência Regional de Saúde Cariri,-39.698883,-6.985781
3,2300804,ANTONINA DO NORTE,230080,5,20,"POLYGON ((-40.03288 -6.73521, -40.03228 -6.734...",Superintendência Regional de Saúde Cariri,-39.981590,-6.756665
4,2301307,ARARIPE,230130,5,20,"POLYGON ((-40.26119 -7.14280, -40.26119 -7.142...",Superintendência Regional de Saúde Cariri,-40.076659,-7.239024


In [53]:
flux = FluxNets(sihsus_df, mun_df1)

In [54]:
flux.build_flux_network(icd_chapter="V")

In [55]:
flux.G

In [56]:
len(flux.G.edges)

247

In [57]:
len(flux.G.nodes)

178